Righnow candidate is : SVR with PCA n_components = 10 with results 0.23706564506631833

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, TheilSenRegressor,RANSACRegressor,HuberRegressor
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test.head(5)

ID  X0 X1  X2 X3 X4 X5 X6 X8  X10  ...   X375  X376  X377  X378  X379  \
0   1  az  v   n  f  d  t  a  w    0  ...      0     0     0     1     0   
1   2   t  b  ai  a  d  b  g  y    0  ...      0     0     1     0     0   
2   3  az  v  as  f  d  a  j  j    0  ...      0     0     0     1     0   
3   4  az  l   n  f  d  z  l  n    0  ...      0     0     0     1     0   
4   5   w  s  as  c  d  y  i  m    0  ...      1     0     0     0     0   

   X380  X382  X383  X384  X385  
0     0     0     0     0     0  
1     0     0     0     0     0  
2     0     0     0     0     0  
3     0     0     0     0     0  
4     0     0     0     0     0  

[5 rows x 377 columns]

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import OneClassSVM

In [5]:
cols = train.columns

In [6]:
lblencod = LabelEncoder()
for col in cols:
    if train[col].dtype == 'object':
        train[col] = lblencod.fit_transform(train[col])

test_cols = test.columns
for col in test_cols:
    if test[col].dtype == 'object':
        test[col] = lblencod.fit_transform(test[col])

In [7]:
train_labels = train.y
del train['y']
train_feats_encode = train

In [8]:
test_feats_encode = test

In [9]:
cellen=['ID', 'X47','X95','X314','X315','X232','X119','X311','X76','X329','X238','X340','X362','X137']

In [10]:
# train_feats_encode = train[cellen].as_matrix()
# test_feats_encode = test[cellen].as_matrix()


In [11]:
train_X, eval_X, train_y, eval_y = train_test_split(train_feats_encode, train_labels)


In [12]:
def check(model , exp=False, train_X=train_X, eval_X=eval_X, train_y=train_y, eval_y=eval_y):
    if exp:
        pred_train = np.exp(model.predict(train_X))
        score_train = r2_score(train_y, pred_train)
        pred_eval = np.exp(model.predict(eval_X))
        score_eval = r2_score(eval_y, pred_eval)
    else:
        pred_train = model.predict(train_X)
        score_train = r2_score(train_y, pred_train)
        pred_eval = model.predict(eval_X)
        score_eval = r2_score(eval_y, pred_eval)
    print(model.__class__.__name__)
    print("r2 train = ", score_train)
    print("r2 eval = ", score_eval)
    print("====================================================\n")
    return score_train, score_eval, pred_train, pred_eval

In [13]:
def eval_model(exp=False):
    # print("TheilSenRegressor ")
    #
    # model_theilsen = TheilSenRegressor(random_state=42)
    # model_theilsen.fit(train_X, train_y)
    # train_theilsen, test_theilsen, pred_train_theilsen, pred_eval_theilsen = check(model_theilsen)
    #
    if exp:
        y = np.log(train_y)
    else:
        y = train_y
    print("RANSACRegressor ")
    model_ransac = RANSACRegressor(random_state=42)
    model_ransac.fit(train_X, y)
    train_ransac, test_ransac, pred_train_ransac, pred_eval_ransac = check(model_ransac, exp)


    print("LinearRegression ")
    model_linear = LinearRegression()
    model_linear.fit(train_X, y)
    train_lr, test_lr, pred_train_lr, pred_eval_lr = check(model_linear, exp)


    model_xgb = XGBRegressor(seed = 0,
      colsample_bytree = 0.7,
      subsample = 0.9,
      eta = 0.005,
      max_depth = 4,
      num_parallel_tree = 1,
      min_child_weight = 1, objective='reg:linear', base_score=np.mean(train_labels))
    model_xgb.fit(train_X, y)
    train_xgb, test_xgb, pred_train_xgb, pred_eval_xgb = check(model_xgb, exp)

    model_gbr = GradientBoostingRegressor()
    model_gbr.fit(train_X, y)
    train_gbr, test_gbr, pred_train_gbr, pred_eval_gbr = check(model_gbr, exp)



    columns=['lr', 'xgb', 'gbr', 'ransac',]

    test_res = (pred_eval_lr, pred_eval_xgb, pred_eval_gbr, pred_eval_ransac)

    train_res = (pred_train_lr, pred_train_xgb, pred_train_gbr, pred_train_ransac)

    train_res_pd = pd.DataFrame(data=np.column_stack(train_res),
                  columns=columns)

    test_res_pd = pd.DataFrame(data=np.column_stack(test_res),
                  columns=columns)
    models = [model_linear, model_xgb, model_gbr, model_ransac]
    return train_res_pd, test_res_pd, models

In [94]:
train_pd, eval_pd, models = eval_model(exp=True)

RANSACRegressor 
RANSACRegressor
r2 train =  0.440106585913
r2 eval =  0.42053027003

LinearRegression 
LinearRegression
r2 train =  0.585988449061
r2 eval =  0.577861871398

XGBRegressor
r2 train =  0.626665876316
r2 eval =  0.61540600477

GradientBoostingRegressor
r2 train =  0.616935568078
r2 eval =  0.609234543911



In [113]:
def frm(dt):
    x = 0. * dt['lr']  + 1. * dt['xgb'] + 0. * dt['gbr'] 
    return x

In [116]:
print("Train score : ", r2_score(train_y, train_pd.mean(axis=1)))
print("eval score : ", r2_score(eval_y, eval_pd.mean(axis=1)))
print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
print("Train score : ", r2_score(train_y, frm(train_pd)))
print("eval score : ", r2_score(eval_y, frm(eval_pd)))
print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
print("Train score : ", r2_score(train_y, train_pd.median(axis=1)))
print("eval score : ", r2_score(eval_y, eval_pd.median(axis=1)))
print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
print("Train score : ", r2_score(train_y, frm(train_pd)))
print("eval score : ", r2_score(eval_y, frm(eval_pd)))

Train score :  0.599574249282
eval score :  0.600047836907
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Train score :  0.626665876316
eval score :  0.61540600477
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Train score :  0.609490324686
eval score :  0.613179374861
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Train score :  0.626665876316
eval score :  0.61540600477


In [108]:
all_models = []
for model in models:
    nme = str(model.__class__.__name__)
    if nme != 'KerasRegressor':
        model.fit(train_feats_encode, np.log(train_labels))
        print(nme)
        print(r2_score(train_labels, np.exp(model.predict(train_feats_encode))))
        all_models.append(model)

LinearRegression
0.594169705436
XGBRegressor
0.630540235593
GradientBoostingRegressor
0.621005122367
RANSACRegressor
0.495371018739


In [54]:
preds = []
for model in models:
    preds.append(np.exp(model.predict(train_feats_encode)))
columns=['lr', 'xgb', 'gbr', 'ransac']
dt = pd.DataFrame(data=np.array(preds).T, columns=columns)
score_final = r2_score(train_labels, frm(dt))
print(score_final)

0.630540235593


In [55]:
preds = []
columns=['lr', 'xgb', 'gbr', 'ransac']

for model in models:
    preds.append(np.exp(model.predict(test_feats_encode)))
    
dt = pd.DataFrame(data=np.array(preds).T, columns=columns)

In [56]:
res = dt[['lr', 'xgb', 'gbr',  'ransac']]

In [57]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32)})


In [58]:
output['y'] = frm(res)

In [59]:
fname = 'results/xgboost/ensemble[%s][%s].csv' % ('ensemble',str(score_final).replace(".", "_"))
output.to_csv(fname, index=False)
print(fname)

results/xgboost/ensemble[ensemble][0_630540235593].csv
